# 전략 POC 01 - 종목발굴을 위한 개인용 파이낸스DB

* POC 01 프로젝트팀의 결과물을 정리해서 공유합니다.
* 목표했던 기능의 일부만 구현된 상태입니다.
* 코드를 이용하실때 패스워드는 변경하여 이용하십시오.

In [1]:
# 이 프로젝트를 실행하기 위해 필요한 준비

# 필요한 모듈 인스톨
    # sudo pip install pymysql
    # sudo pip install PyQuery

# mysql 서버 실행
    # mysql-ctl start

    
# mysql 콘솔 진입
    # mysql-ctl cli
    
# 사용자 생성
    # CREATE USER 'poc1user'@'localhost' IDENTIFIED BY '패스워드'
    # GRANT ALL PRIVILEGES ON * . * TO 'poc1user'@'localhost'
    # FLUSH PRIVILEGES;
    
# 데이터 베이스 생성
    # create database poc1;


In [5]:
import requests
import csv
import os
from datetime import datetime
import pymysql.cursors
import pymysql
import string
from pyquery import PyQuery as pq
import pandas as pd
import time

In [11]:
# DB초기화 및 생성

def init_db():
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            sql = """
                    DROP TABLE IF EXISTS price_day;
                    
                    CREATE TABLE `price_day` (
                      `id` int(11) unsigned NOT NULL AUTO_INCREMENT,
                      `date` date NOT NULL,
                      `stock_code` varchar(20) NOT NULL DEFAULT '',
                      `start` bigint(20) NOT NULL DEFAULT '0',
                      `end` bigint(11) NOT NULL DEFAULT '0',
                      `high` bigint(11) NOT NULL DEFAULT '0',
                      `low` bigint(11) NOT NULL DEFAULT '0',
                      `volume` bigint(11) NOT NULL DEFAULT '0',
                      PRIMARY KEY (`id`),
                      KEY `date_idx` (`date`),
                      KEY `stock_code_idx` (`stock_code`)
                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8;
                    
                    DROP TABLE IF EXISTS rank_and_price;
                    
                    CREATE TABLE `rank_and_price` (
                      `id` bigint(11) unsigned NOT NULL AUTO_INCREMENT,
                      `date` date NOT NULL,
                      `rank` int(11) NOT NULL,
                      `stock_code` varchar(20) NOT NULL DEFAULT '',
                      `stock_name` varchar(20) DEFAULT NULL,
                      `price` bigint(20) DEFAULT NULL,
                      `compare_price` bigint(20) DEFAULT NULL,
                      `up_down_percentage` float DEFAULT NULL,
                      `exchange_volume` bigint(20) DEFAULT NULL,
                      `trade_amount` bigint(20) DEFAULT NULL,
                      `market_capitalization` bigint(20) DEFAULT NULL,
                      `share` bigint(20) DEFAULT NULL,
                      `foreign_hold` bigint(20) DEFAULT '0',
                      PRIMARY KEY (`id`),
                      KEY `date_idx` (`date`),
                      KEY `stock_code_idx` (`stock_code`)
                    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;
                    
                    DROP TABLE IF EXISTS income_statement;
                    
                    CREATE TABLE `income_statement` (
                      `id` bigint(20) NOT NULL AUTO_INCREMENT,
                      `date` datetime DEFAULT NULL,
                      `code` varchar(20) DEFAULT NULL,
                      `sales_revenue` bigint(20) DEFAULT NULL,
                      `cost_of_sales` bigint(20) DEFAULT NULL,
                      `gross_profit` bigint(20) DEFAULT NULL,
                      `selling_and_administrative_expenses` bigint(20) DEFAULT NULL,
                      `operating_income` bigint(20) DEFAULT NULL,
                      `financial_profit` bigint(20) DEFAULT NULL,
                      `finance_income` bigint(20) DEFAULT NULL,
                      `finance_cost` bigint(20) DEFAULT NULL,
                      `other_non_operating_income` bigint(20) DEFAULT NULL,
                      `other_non_operating_revenues` bigint(20) DEFAULT NULL,
                      `other_non_operating_expenses` bigint(20) DEFAULT NULL,
                      `impairment_losses_on_investments_in_associates` bigint(20) DEFAULT NULL,
                      `net_income_or_losses` bigint(20) DEFAULT NULL,
                      `profit` bigint(20) DEFAULT NULL,
                      `other_comprehensive_income_other` bigint(20) DEFAULT NULL,
                      `total_comprehensive_income` bigint(20) DEFAULT NULL,
                      PRIMARY KEY (`id`),
                      KEY `date_idx` (`date`)
                    ) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;
                """
            cursor.execute(sql)
            result = cursor.fetchone()
    finally:
        connection.close()
        
init_db()

In [12]:
# 전체 종목정보를 가져오기
def crawl_rank_and_price(target_date):
    
    directory = "./crawling_raw_data/rank_and_price"
    target_file = "%s/%s.csv" % (directory, target_date)
    target_date_datetype = datetime.strptime(target_date, '%Y%m%d')

    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(target_file):
        with open(target_file) as file:
            textdata = file.read()
            
    else:
        gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
        gen_otp_data = {
            'name':'fileDown',
            'filetype':'csv',
            'url':'MKD/04/0404/04040200/mkd04040200_01',
            'market_gubun':'ALL', # ''ALL':전체, STK': 코스피
            'indx_ind_cd':'',  # '': 전체, ''1001':코스피, 1028' :코스피 200
            'sect_tp_cd':'',
            'schdate':target_date, # '20160408'
            'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
        }
            
        r = requests.post(gen_otp_url, gen_otp_data)
        code = r.content
        
        down_url = 'http://file.krx.co.kr/download.jspx'
        down_data = {
            'code': code,
        }
        
        r = requests.post(down_url, down_data)

        textdata = r.text;

        with open(target_file, 'w+') as f:
            f.write(textdata)


    stock_data = dict()

    # 샘플 데이터
    # 0   1      2    3    4   5     6    7      8            9            10           11           12            13    
    # 순위,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가총액(백만원),시가총액비중(%),상장주식수(천주),외국인 보유주식수,외국인 지분율(%),총카운트
    # 1,005930,삼성전자,"1,246,000","-23,000","-1.8","251,868","315,017","180,756,394","12.35","145,069,337","71,827,823","49.51",2166
    
    reader = csv.reader(textdata.splitlines()[1:], delimiter=',')
    
    for data in reader:
        stock_data['date'] = target_date_datetype
        stock_data['rank'] = data[0]
        stock_data['stock_code'] = data[1]
        stock_data['stock_name'] = data[2]
        stock_data['price'] = safe_convert_string_to_integer(data[3])
        stock_data['compare_price'] = safe_convert_string_to_integer(data[4])
        stock_data['up_down_percentage'] = data[5]
        stock_data['exchange_volume'] = safe_convert_string_to_integer(data[6])
        stock_data['trade_amount'] = safe_convert_string_to_integer(data[7])
        stock_data['market_capitalization'] = safe_convert_string_to_integer(data[8])
        stock_data['market_share'] = data[9]
        stock_data['share'] = safe_convert_string_to_integer(data[10])
        if ( len(data) > 11 ): # 가끔 데이터가 비어있어서 예외처리 필요
            stock_data['foreign_hold'] = safe_convert_string_to_integer(data[11])
        else :
            stock_data['foreign_hold'] = 0
        insert_db_rank_and_price(stock_data)

        
        
# 데이터의 ',' 제거
def safe_convert_string_to_integer(input_string):
    if input_string:
        return input_string.replace(',', '')
    else :
        return 0
    

def insert_db_rank_and_price(data):
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO `rank_and_price` (`date`, `rank`, `stock_code`, `stock_name`, `price`, `compare_price`, `up_down_percentage`, `exchange_volume`, `trade_amount`, `market_capitalization`, `share`, `foreign_hold`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            sql_data = (data['date'], data['rank'], data['stock_code'], data['stock_name'], data['price'], data['compare_price'], data['up_down_percentage'], data['exchange_volume'], data['trade_amount'], data['market_capitalization'], data['share'], data['foreign_hold'])
            cursor.execute(sql, sql_data)
            connection.commit()
    finally:
        connection.close()
    
# rank_and_price 테이블 확인
def select_rank_and_price():
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            sql = "SELECT * FROM rank_and_price"
            cursor.execute(sql)
            result = cursor.fetchall()
            print(result)
    finally:
        connection.close()

        


# 2016년5월4일자 시총순위별로 종목에 대한 기본정보 크롤링
target_date = '20160504'
crawl_rank_and_price(target_date)

# 데이터가 잘 들어갔는지 확인
select_rank_and_price()


[{'up_down_percentage': 2.3, 'id': 1, 'price': 1290000, 'trade_amount': 375425, 'market_capitalization': 184430445, 'exchange_volume': 294055, 'share': 142969337, 'stock_code': '005930', 'stock_name': '삼성전자', 'date': datetime.date(2016, 5, 4), 'rank': 1, 'foreign_hold': 0, 'compare_price': 29000}, {'up_down_percentage': -0.5, 'id': 2, 'price': 61500, 'trade_amount': 42097, 'market_capitalization': 39480791, 'exchange_volume': 684860, 'share': 641964077, 'stock_code': '015760', 'stock_name': '한국전력', 'date': datetime.date(2016, 5, 4), 'rank': 2, 'foreign_hold': 0, 'compare_price': -300}, {'up_down_percentage': -2.5, 'id': 3, 'price': 139000, 'trade_amount': 113214, 'market_capitalization': 30618431, 'exchange_volume': 807340, 'share': 220276479, 'stock_code': '005380', 'stock_name': '현대차', 'date': datetime.date(2016, 5, 4), 'rank': 3, 'foreign_hold': 0, 'compare_price': -3500}, {'up_down_percentage': -0.2, 'id': 4, 'price': 259500, 'trade_amount': 54084, 'market_capitalization': 25260732

In [16]:
# 각 종목 기간 상세 데이터를 가져온기
def crawl_data_day(stock_code, from_date, to_date):
    
    directory = "./crawling_raw_data/day/%s" % (stock_code)
    target_file = "%s/%s-%s_%s.csv" % (directory, from_date, to_date, stock_code)
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(target_file):
        print ('file exist');
        with open(target_file) as file:
            textdata = file.read()
            
    else:
        print('file not exist')
        gen_otp_url = "http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx"
        gen_otp_data = {
            'name':'fileDown',
            'filetype':'csv',
            'url':'MKD/04/0402/04020100/mkd04020100t3_02',
            'isu_cd':stock_code,
            'fromdate':from_date,
            'todate':to_date,
        }
        
        r = requests.post(gen_otp_url, gen_otp_data)
        code = r.text
        
        down_url = 'http://file.krx.co.kr/download.jspx'
        down_data = {
            'code': code,
        }
        
        r = requests.post(down_url, down_data)
        r.encoding = "utf-8-sig"
        
        textdata = r.text;
        
        # save raw data
        with open(target_file, 'w+') as f:
            f.write(textdata)

    stock_data = dict()
    
    # 거래소 샘플 데이터
    # 0       1   2   3        4         5   6   7   8           9 
    # 년/월/일,종가,대비,거래량(주),거래대금(원),시가,고가,저가,시가총액(백만),상장주식수(주)
    # 2015/12/30,"1,260,000","6,000","203,958","257,149,317,000","1,260,000","1,272,000","1,254,000","185,597,165","147,299,337"
    
    # [1:] 구문은 첫번째 라인(헤더)를 스킵하기 위함
    reader = csv.reader(textdata.splitlines()[1:], delimiter=',')
    
    for day_data in reader:
        stock_data['stock_code'] = stock_code
        stock_data['date'] = datetime.strptime(day_data[0], '%Y/%m/%d')
        stock_data['end'] = safe_convert_string_to_integer(day_data[1])
        stock_data['volume'] = safe_convert_string_to_integer(day_data[3])
        stock_data['start'] = safe_convert_string_to_integer(day_data[5])
        stock_data['high'] = safe_convert_string_to_integer(day_data[6])
        stock_data['low'] = safe_convert_string_to_integer(day_data[7])
        insert_db_price_day(stock_data)

        
def insert_db_price_day(data):
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            sql = "INSERT INTO `price_day` (`date`, `stock_code`, `start`, `end`, `high`, `low`, `volume`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            sql_data = (data['date'], data['stock_code'], data['start'], data['end'], data['high'], data['low'], data['volume'])
            cursor.execute(sql, sql_data)
            connection.commit()
    finally:
        connection.close()
        

# 일반 종목코드를 ISIN 표준체계 코드로 변환
def convert_stock_code_basic_to_isin(raw_stock_code):
    isin_stock_code = 'KR7' + raw_stock_code + '00' 
    isin_stock_code = isin_stock_code + calc_checkdigit(isin_stock_code)
    return isin_stock_code


# ISIN의 마지막 체크코드 생성기 
# http://sancs.tistory.com/40
def calc_checkdigit(isin):
    #Convert alpha characters to digits
    isin2 = []
    for char in isin:
        if char.isalpha():
            isin2.append((string.ascii_uppercase.index(char.upper()) + 9 + 1))
        else:
            isin2.append(char)
    #Convert each int into string and join
    isin2 = ''.join([str(i) for i in isin2])
    #Gather every second digit (even)
    even = isin2[::2]
    #Gather the other digits (odd)
    odd = isin2[1::2]
    #If len(isin2) is odd, multiply evens by 2, else multiply odds by 2
    if len(isin2) % 2 > 0:
        even = ''.join([str(int(i)*2) for i in list(even)])
    else:
        odd = ''.join([str(int(i)*2) for i in list(odd)])
    even_sum = sum([int(i) for i in even])
    #then add each single int in both odd and even
    odd_sum = sum([int(i) for i in odd])
    mod = (even_sum + odd_sum) % 10
    return str( (10-mod) % 10 )


def select_price_day():
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            sql = "SELECT * FROM price_day"
            cursor.execute(sql)
            result = cursor.fetchall()
            print(result)
    finally:
        connection.close()
        

# 삼성전자에 대한 2016년 1월 데이터를 크롤링
samsung_stock_code = convert_stock_code_basic_to_isin('005930')  
from_date = '20160101'
to_date = '20160131'
crawl_data_day(samsung_stock_code, from_date, to_date)  

# 데이터가 잘 저장되었는지 확인
select_price_day()

file exist
[{'id': 1, 'date': datetime.date(2016, 1, 29), 'low': 1116000, 'start': 1140000, 'end': 1150000, 'volume': 436983, 'stock_code': 'KR7005930003', 'high': 1150000}, {'id': 2, 'date': datetime.date(2016, 1, 28), 'low': 1139000, 'start': 1164000, 'end': 1145000, 'volume': 315912, 'stock_code': 'KR7005930003', 'high': 1168000}, {'id': 3, 'date': datetime.date(2016, 1, 27), 'low': 1126000, 'start': 1126000, 'end': 1175000, 'volume': 276237, 'stock_code': 'KR7005930003', 'high': 1175000}, {'id': 4, 'date': datetime.date(2016, 1, 26), 'low': 1136000, 'start': 1155000, 'end': 1137000, 'volume': 152399, 'stock_code': 'KR7005930003', 'high': 1157000}, {'id': 5, 'date': datetime.date(2016, 1, 25), 'low': 1156000, 'start': 1172000, 'end': 1162000, 'volume': 159913, 'stock_code': 'KR7005930003', 'high': 1176000}, {'id': 6, 'date': datetime.date(2016, 1, 22), 'low': 1145000, 'start': 1145000, 'end': 1168000, 'volume': 147528, 'stock_code': 'KR7005930003', 'high': 1168000}, {'id': 7, 'date'

In [19]:
# 매일매일 전체 종목에 대한 주가정보를 크롤링 하기
def crawl_all_stock_day(target_date):
    stock_code_list = get_stock_code_list(target_date)

    # 테스트 이므로 시총 상위 10종목만 크롤링
    for row in stock_code_list[:10]:
        krx_stock_code = convert_stock_code_basic_to_isin(row['stock_code']);
        print ('%s - %s' % (krx_stock_code, row['stock_name']) )
        
        crawl_data_day(krx_stock_code, from_date, to_date)


def get_stock_code_list(target_date):
    connection = pymysql.connect(host='localhost',
                                 user='poc1user',
                                 password='패스워드',
                                 db='poc1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    try:
        with connection.cursor() as cursor:
            sql = "SELECT stock_code, stock_name FROM rank_and_price"
            cursor.execute(sql)
            result = cursor.fetchall()
            return result
    finally:
        connection.close()


# 전체 종목 크롤링 시작
target_date = '20160504'
crawl_all_stock_day(target_date)

# 데이터가 잘 저장되었는지 확인
select_price_day()

KR7005930003 - 삼성전자
file exist
KR7015760002 - 한국전력
file exist
KR7005380001 - 현대차
file exist
KR7012330007 - 현대모비스
file exist
KR7090430000 - 아모레퍼시픽
file exist
KR7028260008 - 삼성물산
file exist
KR7035420009 - NAVER
file exist
KR7005935002 - 삼성전자우
file exist
KR7032830002 - 삼성생명
file exist
KR7000660001 - SK하이닉스
file exist
[{'id': 1, 'date': datetime.date(2016, 1, 29), 'low': 1116000, 'start': 1140000, 'end': 1150000, 'volume': 436983, 'stock_code': 'KR7005930003', 'high': 1150000}, {'id': 2, 'date': datetime.date(2016, 1, 28), 'low': 1139000, 'start': 1164000, 'end': 1145000, 'volume': 315912, 'stock_code': 'KR7005930003', 'high': 1168000}, {'id': 3, 'date': datetime.date(2016, 1, 27), 'low': 1126000, 'start': 1126000, 'end': 1175000, 'volume': 276237, 'stock_code': 'KR7005930003', 'high': 1175000}, {'id': 4, 'date': datetime.date(2016, 1, 26), 'low': 1136000, 'start': 1155000, 'end': 1137000, 'volume': 152399, 'stock_code': 'KR7005930003', 'high': 1157000}, {'id': 5, 'date': datetime.date(201

# 아이투자 재무데이터 크롤링

In [ ]:
# 아이디와 패스워드는 수정필요

# 아이투자 로그인 
def login_itooza():
    login_data = {
        'hdnRetUrl': '',
        'txtUserId': '아이디',
        'x': 0,
        'y': 0,
        'txtPassword': '패스워드'
    }
    
    logined_session = requests.session()
    logined_session.post('https://login.itooza.com/login_process.php', login_data)
    return logined_session


# 제무상태표 크롤링
def crawl_financial_position(stock_code, logined_session):
    directory = "./crawling_raw_data/financial_position"
    target_file = "%s/%s.html" % (directory, stock_code)

    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(target_file):
        print ('file exist');
        with open(target_file) as file:
            textdata = file.read()
        
    else :
        print ('file not exist');
        # http://www.itooza.com/vclub/y10_page.php?cmp_cd=055550&mode=db&ss=10&sv=1&lsmode=1&lkmode=1&accmode=1
        profit_and_loss_html = logined_session.get( 'http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=1&lsmode=1&lkmode=1&accmode=1' % stock_code )
        profit_and_loss_html.encoding = "euc-kr"
        textdata = profit_and_loss_html.text
        
        with open(target_file, 'w+') as f:
            f.write(textdata)
    

# 손익계산서 크롤링 (옵션 : 국제회계, 연결 재무제표, 분기)
def crawl_income_statement(stock_code, logined_session):
    directory = "./crawling_raw_data/income_statement"
    target_file = "%s/%s.html" % (directory, stock_code)

    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(target_file):
        print ('file exist');
        with open(target_file) as file:
            textdata = file.read()
        
    else :
        print ('file not exist');
        # http://www.itooza.com/vclub/y10_page.php?cmp_cd=055550&mode=db&ss=10&sv=2&lsmode=1&lkmode=1&accmode=1
        profit_and_loss_html = logined_session.get( 'http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=2&lsmode=1&lkmode=1&accmode=1' % stock_code )
        profit_and_loss_html.encoding = "euc-kr"
        textdata = profit_and_loss_html.text
        
        with open(target_file, 'w+') as f:
            f.write(textdata)
    

# 현금흐름표 크롤링
def crawl_cash_flow(stock_code, logined_session):
    directory = "./crawling_raw_data/cash_flow"
    target_file = "%s/%s.html" % (directory, stock_code)

    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(target_file):
        print ('file exist');
        with open(target_file) as file:
            textdata = file.read()
        
    else :
        print ('file not exist');
        # http://www.itooza.com/vclub/y10_page.php?cmp_cd=055550&mode=db&ss=10&sv=1&lsmode=4&lkmode=1&accmode=1
        profit_and_loss_html = logined_session.get( 'http://www.itooza.com/vclub/y10_page.php?cmp_cd=%s&mode=db&ss=10&sv=4&lsmode=1&lkmode=1&accmode=1' % stock_code )
        profit_and_loss_html.encoding = "euc-kr"
        textdata = profit_and_loss_html.text
        
        with open(target_file, 'w+') as f:
            f.write(textdata)

            
# 전체 재무상태표 크롤링
def crawl_all_financial_position(target_date):
    stock_code_list = get_stock_code_list(target_date)
    logined_session = login_itooza()

    for row in stock_code_list:
        print ('%s - %s' % (row['stock_code'], row['stock_name']) )
        
        crawl_financial_position(row['stock_code'], logined_session)
        time.sleep(1)

# 전체
def crawl_all_income_statement(target_date):
    stock_code_list = get_stock_code_list(target_date)
    logined_session = login_itooza()

    for row in stock_code_list:
        print ('%s - %s' % (row['stock_code'], row['stock_name']) )
        
        crawl_income_statement(row['stock_code'], logined_session)
        time.sleep(1)

# 전체 현금흐름표 크롤링
def crawl_all_cash_flow(target_date):
    stock_code_list = get_stock_code_list(target_date)
    logined_session = login_itooza()

    for row in stock_code_list:
        print ('%s - %s' % (row['stock_code'], row['stock_name']) )
        
        crawl_cash_flow(row['stock_code'], logined_session)
        time.sleep(1)


# 시간이 오래 걸리는 작업이므로 실행하지는 않겠습니다.        
# logined_session = login_itooza()
# crawl_income_statement('005930', logined_session)

# target_date = '20160504'
# crawl_all_financial_position(target_date)
# crawl_all_income_statement(target_date)
# crawl_all_cash_flow(target_date)